# Bank Marketing Data - A Multilayer Perceptron (MLP) Approach

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

You can download the data from the [University of California, Irvine's website](http://archive.ics.uci.edu/ml/datasets/Bank+Marketing).

The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).

### Install and Load Libraries

In [64]:
#!pip install wandb

In [65]:
# Imports
import logging
import wandb
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from imblearn.metrics import geometric_mean_score

from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import os

# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import cross_val_score
# imblearn

In [66]:
# Login to Weights & Biases
#!wandb login --relogin

# 1.0 Training



## 1.0.2 Holdout Configuration

In [67]:
# global variables

# ratio used to split train and validation data
val_size = 0.30
# seed used to reproduce purposes
seed = 41
# reference (column) to stratify the data
stratify = "y"
# name of the input artifact
artifact_input_name = "mlops_ivan/bank_mlp/train.csv:latest"
# type of the artifact
artifact_type = "Train"

In [68]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

# initiate the wandb project
run = wandb.init(project="bank_mlp", entity="mlops_ivan", job_type="train")

logger.info("Downloading and reading train artifact")
local_path = run.use_artifact(artifact_input_name).file()
df_train = pd.read_csv(local_path)


17-07-2022 10:40:15 Downloading and reading train artifact


In [69]:
# Spliting train.csv into train and validation dataset
logger.info("Spliting data into train/val")

# split-out train/validation dataset
train, val = train_test_split(  df_train,
                                test_size=val_size,
                                random_state=seed,
                                shuffle=True
                                )

17-07-2022 10:40:16 Spliting data into train/val


In [70]:
logger.info("train: {}".format(train.shape))
logger.info("val: {}".format(val.shape))

17-07-2022 10:40:16 train: (22152, 17)
17-07-2022 10:40:16 val: (9495, 17)


## 1.1 Data preparation

### 1.1.1 Outlier Removal

In [71]:
logger.info("Outlier Removal")
# temporary variable
x = train.select_dtypes("int64").copy()

# identify outlier in the dataset
lof = LocalOutlierFactor()
outlier = lof.fit_predict(x)
mask = outlier != -1

17-07-2022 10:40:17 Outlier Removal


In [72]:
logger.info("train shape [original]: {}".format(train.shape))
logger.info("train shape [outlier removal]: {}".format(train.loc[mask,:].shape))

17-07-2022 10:40:17 train shape [original]: (22152, 17)
17-07-2022 10:40:17 train shape [outlier removal]: (21634, 17)


In [73]:
# AVOID data leakage and you should not do this procedure in the preprocessing stage
# Note that we did not perform this procedure in the validation set
train = train.loc[mask,:].copy()

### 1.1.2 Balancing Data

Reference:
https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

In [74]:
train.y.value_counts()

no     19250
yes     2384
Name: y, dtype: int64

In [75]:
# Return dataframe balanced: oversample and undersample
def resaple_bal(df, seed): 
  
  x_train = df.loc[:, df.columns != 'y']
  y_train = df.loc[:, df.columns == 'y']  
  oversample = SMOTENC(categorical_features=[1, 2, 3, 4, 6, 7, 8, 10, 15],  sampling_strategy = 0.4, random_state=seed) # 0.4
  undersample = RandomUnderSampler(sampling_strategy=0.7, random_state=seed)
  x_train, y_train = oversample.fit_resample(x_train, y_train)
  x_train, y_train = undersample.fit_resample(x_train, y_train)
  df_o = pd.concat([x_train, y_train], axis=1)
  df_o = df_o.sample(frac=1, random_state=seed).reset_index(drop=True)
  return df_o


In [76]:
logger.info("Balance data")
train = resaple_bal( train, seed)

17-07-2022 10:40:17 Balance data


In [77]:
# for validation purposes
# train.loc[0:10, 'y']

In [78]:
# for validation purposes
train.y.value_counts()

no     11000
yes     7700
Name: y, dtype: int64

### 1.1.3 Encoding Target Variable

In [79]:
logger.info("Encoding Target Variable")
# define a categorical encoding for target variable
le = LabelEncoder()

# fit and transform y_train
train.loc[:, 'y'] = le.fit_transform(train.loc[:, 'y'])

# transform y_test (avoiding data leakage)
val.loc[:, 'y'] = le.transform(val.loc[:, 'y'])

val = val.reset_index(drop=True)

logger.info("Classes [0, 1]: {}".format(le.inverse_transform([0, 1])))

17-07-2022 10:40:19 Encoding Target Variable
17-07-2022 10:40:19 Classes [0, 1]: ['no' 'yes']


### 1.1.4 Data split-out features and label

In [80]:
x_train = train.loc[:, train.columns != 'y']
y_train = train.loc[:, train.columns == 'y']  
x_val = val.loc[:, val.columns != 'y']
y_val = val.loc[:, val.columns == 'y']  

## 1.2 Data full-pipeline

### 1.2.1 Feature Extractor

In [81]:
def copy_exception(feature_names, exceptions):
  # this functios return the itens in the feature_names except exceptions
  # if exceptions do not exist in feature_names it returns all feature_names
  # Imput: feature_names, exceptions: list
  # Output: list
  for exception in exceptions:
    try:
      feature_names.remove(exception)
    except ValueError:
      pass
    features_selected = feature_names
  return features_selected

In [82]:
f_sel = copy_exception(x_train.select_dtypes("object").columns.to_list(), ["contact", "day"])
print(f_sel)

['job', 'marital', 'education', 'default', 'housing', 'loan', 'month', 'poutcome']


In [83]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    # Class Constructor
    def __init__(self, feature_names):
      #exceptions = ["contact", "day", 'default', 'loan', 'marital']
      exceptions = ['default', 'loan']
      self.feature_names = copy_exception(feature_names, exceptions) 
      #self.feature_names = feature_names


    # Return self nothing else to do here
    def fit(self, X, y=None):
        return self

    # Method that describes what this custom transformer need to do
    def transform(self, X, y=None):
        return X[self.feature_names]

# Handling categorical features
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes one boolean as its argument
    def __init__(self, new_features=True, colnames=None):
        self.new_features = new_features
        self.colnames = colnames

    # Return self nothing else to do here
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self):
        return self.colnames.tolist()

    # Transformer method we wrote for this transformer
    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)

        # Remove white space in categorical features
        df = df.apply(lambda row: row.str.strip())

        # customize feature?
        # How can I identify what needs to be modified? EDA!!!!
        if self.new_features:
            # df['job'] = df['job']
            # Combine similar jobs into categiroes
            df['job'] = df['job'].replace(['management', 'admin.'], 'white-collar')
            df['job'] = df['job'].replace(['services','housemaid'], 'pink-collar')
            df['job'] = df['job'].replace(['retired', 'student', 'unemployed', 'unknown'], 'other')

            # minimize the cardinality poutcome
            df['poutcome'] = df['poutcome'].replace(['other'] , 'unknown')

            # Drop 'contact', as every participant has been contacted. 
            # df.drop('contact', axis=1, inplace=True)

            # day  : last contact day of the month
            # month: last contact month of year
            # Drop 'month' as they don't have any intrinsic meaning
            # df.drop('month', axis=1, inplace=True)
            df['month'] = df['month'].replace(['feb', 'mar', 'jan'], '1st')
            df['month'] = df['month'].replace(['may', 'jun', 'apr'], '2st')
            df['month'] = df['month'].replace(['jul', 'aug', 'sep'], '3st')
            df['month'] = df['month'].replace(['nov', 'oct', 'dec'], '4st')         

        # update column names
        self.colnames = df.columns

        return df
        
# transform numerical features
class NumericalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes a model parameter as its argument
    # model 0: minmax
    # model 1: standard
    # model 2: without scaler
    def __init__(self, model=0, colnames=None):
        self.model = model
        self.colnames = colnames
        self.scaler = None

    # Fit is used only to learn statistical about Scalers
    def fit(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        # minmax
        if self.model == 0:
            self.scaler = MinMaxScaler()
            self.scaler.fit(df)
        # standard scaler
        elif self.model == 1:
            self.scaler = StandardScaler()
            self.scaler.fit(df)
        return self

    # return columns names after transformation
    def get_feature_names_out(self):
        return self.colnames

    # Transformer method we wrote for this transformer
    # Use fitted scalers
    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)

        # day  : last contact day of the month                  # adicionado
        # Drop 'day' as they don't have any intrinsic meaning
        # df.drop('day', axis=1, inplace=True)

        # Map padys=-1 into a large value (10000 is used) to indicate that it is so far in the past that it has no effect
        df.loc[df['pdays'] == -1, 'pdays'] = 10000

        # update columns name
        self.colnames = df.columns.tolist()

        # minmax
        if self.model == 0:
            # transform data
            df = self.scaler.transform(df)
        elif self.model == 1:
            # transform data
            df = self.scaler.transform(df)
        else:
            df = df.values

        return df

In [84]:
# for validation purposes
fs = FeatureSelector(x_train.select_dtypes("object").columns.to_list())
df = fs.fit_transform(x_train)
df.head()

,job,marital,education,housing,contact,month,poutcome
0,admin.,married,secondary,no,cellular,aug,success
1,technician,married,tertiary,no,cellular,oct,success
2,technician,married,secondary,no,unknown,may,unknown
3,admin.,divorced,secondary,no,cellular,jul,unknown
4,technician,married,tertiary,yes,cellular,jul,unknown


In [85]:
# for validation purposes
fs = FeatureSelector(x_train.select_dtypes("int64").columns.to_list())
df = fs.fit_transform(x_train)
df.head()

,age,balance,day,duration,campaign,pdays,previous
0,38,261,6,311,1,180,5
1,33,620,11,329,1,119,2
2,30,764,26,150,2,-1,0
3,34,1927,18,142,2,-1,0
4,47,1430,24,120,11,-1,0


### 1.2.2 Handling Categorical Features

In [86]:
x_train.month.unique()

array(['aug', 'oct', 'may', 'jul', 'feb', 'sep', 'jan', 'jun', 'apr',
       'mar', 'nov', 'dec'], dtype=object)

In [87]:
# for validation purposes
fs = FeatureSelector(x_train.select_dtypes("object").columns.to_list())
df = fs.fit_transform(x_train)
ct = CategoricalTransformer(new_features=True,colnames=df.columns.tolist())
df_cat = ct.fit_transform(df)
df_cat.head()

,job,marital,education,housing,contact,month,poutcome
0,white-collar,married,secondary,no,cellular,3st,success
1,technician,married,tertiary,no,cellular,4st,success
2,technician,married,secondary,no,unknown,2st,unknown
3,white-collar,divorced,secondary,no,cellular,3st,unknown
4,technician,married,tertiary,yes,cellular,3st,unknown


In [88]:
# check the cardinality before and after transformation
x_train.select_dtypes("object").apply(pd.Series.nunique)

job          12
marital       3
education     4
default       2
housing       2
loan          2
contact       3
month        12
poutcome      4
dtype: int64

In [89]:
# check the cardinality before and after transformation
df_cat.apply(pd.Series.nunique)

job          7
marital      3
education    4
housing      2
contact      3
month        4
poutcome     3
dtype: int64

### 1.2.3 Handling Numerical Features

In [90]:
# for validation purposes
fs = FeatureSelector(x_train.select_dtypes("int64").columns.to_list())
df = fs.fit_transform(x_train)
nt = NumericalTransformer(model=1)
df_num = nt.fit_transform(df)
df_num

array([[-2.66982623e-01, -4.28621687e-01, -1.19615979e+00, ...,
        -5.41670969e-01,  1.36146011e+00,  2.29313683e+00],
       [-7.28941564e-01, -3.00784707e-01, -5.64711037e-01, ...,
        -5.41670969e-01,  7.46471852e-01,  7.05769924e-01],
       [-1.00611693e+00, -2.49507478e-01,  1.32963523e+00, ...,
        -1.66124212e-01,  1.00364488e+02, -3.52474680e-01],
       ...,
       [ 4.72151682e-01, -3.79481009e-01, -3.12131535e-01, ...,
        -5.41670969e-01,  1.00364488e+02, -3.52474680e-01],
       [ 1.21128599e+00, -2.31702885e-01,  1.83479423e+00, ...,
        -5.41670969e-01,  1.00364488e+02, -3.52474680e-01],
       [ 1.01927411e-02, -5.16932470e-01, -1.32244954e+00, ...,
        -1.66124212e-01,  1.00364488e+02, -3.52474680e-01]])

In [91]:
# for validation purposes
# return columns name
nt.get_feature_names_out()

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

### 1.2.4 Data Preparation Pipeline

In [92]:
# model = 0 (min-max), 1 (z-score), 2 (without normalization)
numerical_model = 2

# Categrical features to pass down the categorical pipeline
categorical_features = x_train.select_dtypes("object").columns.to_list() 

# Numerical features to pass down the numerical pipeline
numerical_features = x_train.select_dtypes("int64").columns.to_list()

# Defining the steps for the categorical pipeline
categorical_pipeline = Pipeline(steps=[('cat_selector', FeatureSelector(categorical_features)),
                                       ('imputer_cat', SimpleImputer(strategy="most_frequent")),
                                       ('cat_transformer', CategoricalTransformer(colnames=categorical_features)),
                                       # ('cat_encoder','passthrough'
                                       ('cat_encoder', OneHotEncoder(sparse=False, drop="if_binary"))
                                       ]
                                ) # drop="first"

# Defining the steps in the numerical pipeline
numerical_pipeline = Pipeline(steps=[('num_selector', FeatureSelector(numerical_features)),
                                     ('imputer_num', SimpleImputer(strategy="median")),
                                     ('num_transformer', NumericalTransformer(numerical_model, 
                                                                              colnames=numerical_features))])

# Combine numerical and categorical pieplines into one full big pipeline horizontally
full_pipe = FeatureUnion(transformer_list=[('cat_pipeline', categorical_pipeline),
                                                             ('num_pipeline', numerical_pipeline)]
                                           )

In [93]:
x_train_encoded = full_pipe.fit_transform(x_train)
x_val_encoded = full_pipe.transform(x_val)

# Transform the data
# cat_names is a numpy array
cat_names = full_pipe.get_params()["cat_pipeline"][3].get_feature_names_out().tolist()
# num_names is a list
num_names = full_pipe.get_params()["num_pipeline"][2].get_feature_names_out()
# Transform the data
x_train_encoded = pd.DataFrame(x_train_encoded,columns = cat_names + num_names)
x_val_encoded = pd.DataFrame(x_val_encoded ,columns = cat_names + num_names)

In [94]:
train_encoded = pd.concat([x_train_encoded, y_train], axis=1)
val_encoded = pd.concat([x_val_encoded, y_val], axis=1)

In [95]:
# for validation purposes
train_encoded #.head()

,job_blue-collar,job_entrepreneur,job_other,job_pink-collar,job_self-employed,job_technician,job_white-collar,marital_divorced,marital_married,marital_single,...,poutcome_success,poutcome_unknown,age,balance,day,duration,campaign,pdays,previous,y
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,38.0,261.0,6.0,311.0,1.0,180.0,5.0,1
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,33.0,620.0,11.0,329.0,1.0,119.0,2.0,1
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,30.0,764.0,26.0,150.0,2.0,10000.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,34.0,1927.0,18.0,142.0,2.0,10000.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,47.0,1430.0,24.0,120.0,11.0,10000.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18695,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,35.0,3446.0,12.0,1002.0,3.0,56.0,2.0,1
18696,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,59.0,4248.0,15.0,214.0,1.0,91.0,4.0,1
18697,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,46.0,399.0,13.0,353.0,1.0,10000.0,0.0,0
18698,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,54.0,814.0,30.0,95.0,1.0,10000.0,0.0,0


In [96]:
val_encoded

,job_blue-collar,job_entrepreneur,job_other,job_pink-collar,job_self-employed,job_technician,job_white-collar,marital_divorced,marital_married,marital_single,...,poutcome_success,poutcome_unknown,age,balance,day,duration,campaign,pdays,previous,y
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,27.0,1.0,2.0,261.0,8.0,10000.0,0.0,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,49.0,23.0,23.0,247.0,1.0,10000.0,0.0,0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,33.0,1278.0,6.0,649.0,1.0,10000.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,59.0,266.0,30.0,147.0,2.0,10000.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,38.0,-3.0,20.0,251.0,2.0,101.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9490,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,31.0,1306.0,28.0,217.0,5.0,10000.0,0.0,0
9491,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,43.0,463.0,3.0,268.0,1.0,10000.0,0.0,0
9492,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,28.0,708.0,6.0,339.0,5.0,10000.0,0.0,0
9493,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,27.0,220.0,19.0,480.0,1.0,10000.0,0.0,0


In [97]:
# saving transformed train data to csv
train_encoded.to_csv("encoded_train_data.csv", index=False)
val_encoded.to_csv("encoded_val_data.csv", index=False)

run = wandb.init(project="bank_mlp", entity="mlops_ivan", job_type="encode_data")

In [98]:
# Uploading Transformed Data
# Create a new artifact and configure with the necessary arguments
artifact = wandb.Artifact(name="encoded_train_data.csv",
                          type="encoded_data",
                          description="Upload encoded Train Data")

artifact.add_file("encoded_train_data.csv")
run.log_artifact(artifact)

# Uploading Transformed Data
# Create a new artifact and configure with the necessary arguments
artifact = wandb.Artifact(name="encoded_val_data.csv",
                          type="encoded_data",
                          description="Upload encoded Val Data")

artifact.add_file("encoded_val_data.csv")
run.log_artifact(artifact)

In [99]:
run.finish()

os.remove('encoded_train_data.csv')
os.remove('encoded_val_data.csv')

## 1.6 Hyperparameter Tuning

## 4.1 Monitoring a neural network

In [10]:
import wandb
import logging
from wandb.keras import WandbCallback
import pandas as pd
import numpy as np
import os

from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from imblearn.metrics import geometric_mean_score

# os.environ["CUDA_VISIBLE_DEVICES"]="-1"
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.models import Sequential
from keras import callbacks
# from keras.wrappers.scikit_learn import KerasClassifier
# from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [11]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'train_test.ipynb'

In [12]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

Better Genaralization

1. Regularization L2 ok
2. Penalize large weights 
3. weight constraint
4. Dropout ok
5. Noise ok (oversample)
6. Early stoping ok

In [13]:
# Default values for hyperparameters
defaults = dict(layer_1 = 32,
                layer_2 = 32,
                learn_rate = 0.001,
                batch_size = 512,
                epoch = 300,
                dropout1 = 0.9,
                dropout2 = 0.9,
                l2_1 = 0.0,
                l2_2 = 0.0,
                gradient_cliping = 2,
                bath_norm = 0.5)


In [14]:
run = wandb.init(project="bank_mlp", entity="mlops_ivan", config= defaults, name="run_01")
config = wandb.config

In [15]:
# Downloading and reading encoded train val artifact
logger.info("Downloading and reading encoded train artifact")
local_path = run.use_artifact("encoded_train_data.csv:latest").file()
train_encoded = pd.read_csv(local_path)

logger.info("Downloading and reading encoded val artifact")
local_path = run.use_artifact("encoded_val_data.csv:latest").file()
val_encoded = pd.read_csv(local_path)

x_train_encoded = train_encoded.loc[:, train_encoded.columns != 'y']
y_train = train_encoded.loc[:, train_encoded.columns == 'y']  
x_val_encoded = val_encoded.loc[:, val_encoded.columns != 'y']
y_val = val_encoded.loc[:, val_encoded.columns == 'y'] 

17-07-2022 15:04:59 Downloading and reading encoded train artifact
17-07-2022 15:05:00 Downloading and reading encoded val artifact


In [16]:
# Implemented in preprocess step
# oversample = SMOTE(sampling_strategy = 0.7, random_state=seed)
# x_train_encoded, y_train = oversample.fit_resample(x_train_encoded, y_train)

In [18]:
# Instantiate a simple classification model
model = tf.keras.Sequential([
  layers.Dense( config.layer_1,
                activation=tf.nn.relu,
                kernel_initializer='he_uniform',
                kernel_regularizer = tf.keras.regularizers.L2(l=config.l2_1),
                dtype='float64'),
  layers.Dropout(config.dropout1),
  layers.BatchNormalization(),
  layers.Dense( config.layer_2,
                activation=tf.nn.relu,
                kernel_initializer='he_uniform',
                kernel_regularizer = tf.keras.regularizers.L2(l=config.l2_2),
                dtype='float64'),  
  layers.Dropout(config.dropout2),
  layers.BatchNormalization(),
  layers.Dense(1, activation=tf.nn.sigmoid, dtype='float64')
])

# Instantiate a logistic loss function that expects integer targets.
loss = tf.keras.losses.BinaryCrossentropy()

# Instantiate an accuracy metric.
# metric = tf.keras.metrics.BinaryAccuracy()
# metric = tf.keras.metrics.BinaryAccuracy()
metric = tfa.metrics.F1Score(num_classes=1, threshold=0.5)

# Instantiate an optimizer.

optimizer = tf.keras.optimizers.Adam( learning_rate=config.learn_rate,
                                      clipvalue= config.gradient_cliping
                                    )


# configure the optimizer, loss, and metrics to monitor.
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [19]:
early_stopping = callbacks.EarlyStopping(
                monitor='val_loss',   # 'loss'             
                min_delta=0.001, # Minimum change
                patience=100, # Number of epochs with no improvement
                verbose=1, # 
                mode='auto', # "auto", "min", "max"
                baseline=None, # Baseline value for the monitored quantity
                restore_best_weights=True) # restore weights epoch with best value

In [20]:
%%wandb
# Add WandbCallback() to the fit function
model.fit(x= x_train_encoded , # full_pipe.transform(x_train)
          y= y_train,
          batch_size=config.batch_size,
          epochs=config.epoch,
          validation_data=( x_val_encoded, y_val), # full_pipe.transform(x_val_bal)
          callbacks=[WandbCallback(log_weights=True), early_stopping],
          verbose=1)

# predict
y_pred = np.rint(model.predict(x_val_encoded)).ravel()

# Evaluation Metrics
logger.info("Evaluation metrics")
fbeta = fbeta_score(y_val.values.ravel(), y_pred, beta=1, zero_division=1)
precision = precision_score(y_val.values.ravel(), y_pred, zero_division=1)
recall = recall_score(y_val.values.ravel(), y_pred, zero_division=1)
acc = accuracy_score(y_val.values.ravel(), y_pred)
g_mean = geometric_mean_score(y_val.values.ravel(), y_pred)

logger.info("Accuracy: {}".format(acc))
logger.info("Precision: {}".format(precision))
logger.info("Recall: {}".format(recall))
logger.info("F1: {}".format(fbeta))
logger.info("G-Mean: {}".format(g_mean))

run.summary["Accuracy"] = acc
run.summary["Precision"] = precision
run.summary["Recall"] = recall
run.summary["F1"] = fbeta
run.summary["G-Mean"] = g_mean  

Epoch 1/300
37/37 [==============================] - 1s 7ms/step - loss: 0.8026 - f1_score: 0.3095 - val_loss: 0.6738 - val_f1_score: 0.1379 - _timestamp: 1658081125.0000 - _runtime: 33.0000
Epoch 2/300
37/37 [==============================] - 0s 3ms/step - loss: 0.7682 - f1_score: 0.2627 - val_loss: 0.5873 - val_f1_score: 0.1063 - _timestamp: 1658081126.0000 - _runtime: 34.0000
Epoch 3/300
37/37 [==============================] - 0s 3ms/step - loss: 0.7381 - f1_score: 0.2483 - val_loss: 0.5634 - val_f1_score: 0.0930 - _timestamp: 1658081126.0000 - _runtime: 34.0000
Epoch 4/300
37/37 [==============================] - 0s 3ms/step - loss: 0.7223 - f1_score: 0.2312 - val_loss: 0.5589 - val_f1_score: 0.0786 - _timestamp: 1658081126.0000 - _runtime: 34.0000
Epoch 5/300
37/37 [==============================] - 0s 2ms/step - loss: 0.7088 - f1_score: 0.2301 - val_loss: 0.5599 - val_f1_score: 0.0760 - _timestamp: 1658081126.0000 - _runtime: 34.0000
Epoch 6/300
37/37 [==========================

17-07-2022 15:05:52 Evaluation metrics
17-07-2022 15:05:52 Accuracy: 0.8818325434439178
17-07-2022 15:05:52 Precision: 0.5122985581000848
17-07-2022 15:05:52 Recall: 0.5247610773240661
17-07-2022 15:05:52 F1: 0.5184549356223177
17-07-2022 15:05:52 G-Mean: 0.6989984625439906


In [21]:
run.finish()

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
f1_score,▃▂▁▂▃▃▄▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇█▇████
loss,█▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_f1_score,▁▁▁▂▃▃▄▅▆▆▆▆▇▆▇▇▇▇▇▇▇█████▇▇▇▇██████████
val_loss,████▇▇▇▆▅▄▄▄▄▄▄▄▅▄▄▃▃▃▄▄▄▃▃▂▄▂▄▁▃▂▁▂▃▁▂▁
Accuracy,0.88183
F1,0.51845
G-Mean,0.699
Precision,0.5123
Recall,0.52476
best_epoch,289


## 4.2 Sweeps

In [23]:
def train_func():

     # The sweep calls this function with each set of hyperparameters
    # Default values for hyperparameters
    defaults = dict(layer_1 = 32,
                layer_2 = 32,
                learn_rate = 0.001,
                batch_size = 512,
                epoch = 300,
                dropout1 = 0.9,
                dropout2 = 0.9,
                l2_1 = 0.0,
                l2_2 = 0.0,
                gradient_cliping = 2,
                bath_norm = 1)
    
    # Initialize a new wandb run
    wandb.init(project="bank_mlp", entity="mlops_ivan", config= defaults)
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
    # Instantiate a simple classification model
# Instantiate a simple classification model
    if config.bath_norm==1:
        model = tf.keras.Sequential([
        layers.Dense( config.layer_1,
                        activation=tf.nn.relu,
                        kernel_initializer='he_uniform',
                        kernel_regularizer = tf.keras.regularizers.L2(l=config.l2_1),
                        dtype='float64'),
        layers.Dropout(config.dropout1),
        layers.BatchNormalization(),
        layers.Dense( config.layer_2,
                        activation=tf.nn.relu,
                        kernel_initializer='he_uniform',
                        kernel_regularizer = tf.keras.regularizers.L2(l=config.l2_2),
                        dtype='float64'),  
        layers.Dropout(config.dropout2),
        layers.BatchNormalization(),
        layers.Dense(1, activation=tf.nn.sigmoid, dtype='float64')
        ])
    else:
        model = tf.keras.Sequential([
        layers.Dense( config.layer_1,
                        activation=tf.nn.relu,
                        kernel_initializer='he_uniform',
                        kernel_regularizer = tf.keras.regularizers.L2(l=config.l2_1),
                        dtype='float64'),
        layers.Dropout(config.dropout1),
        layers.Dense( config.layer_2,
                        activation=tf.nn.relu,
                        kernel_initializer='he_uniform',
                        kernel_regularizer = tf.keras.regularizers.L2(l=config.l2_2),
                        dtype='float64'),  
        layers.Dropout(config.dropout2),
        layers.Dense(1, activation=tf.nn.sigmoid, dtype='float64')
        ])
    
    
    # Instantiate a logistic loss function that expects integer targets.
    loss = tf.keras.losses.BinaryCrossentropy()

    # Instantiate an accuracy metric.
    # metric = tf.keras.metrics.BinaryAccuracy()

    metric = tfa.metrics.F1Score(num_classes=1, threshold=0.5)

    # Instantiate an optimizer.
    if config.gradient_cliping==0.0:
        optimizer = tf.keras.optimizers.Adam(learning_rate=config.learn_rate)
    else:
        optimizer = tf.keras.optimizers.Adam( 
                                        learning_rate=config.learn_rate,
                                        clipvalue= config.gradient_cliping
                                        )

    # configure the optimizer, loss, and metrics to monitor.
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    early_stopping = callbacks.EarlyStopping(
                monitor='val_loss',   # 'loss'             
                min_delta=0.001, # Minimum change
                patience=100, # Number of epochs with no improvement
                verbose=1, # 
                mode='auto', # "auto", "min", "max"
                baseline=None, # Baseline value for the monitored quantity
                restore_best_weights=True) # restore weights epoch with best value

    model.fit(x= x_train_encoded , # full_pipe.transform(x_train)
          y= y_train,
          batch_size=config.batch_size,
          epochs=config.epoch,
          validation_data=( x_val_encoded, y_val), # full_pipe.transform(x_val)
          callbacks=[WandbCallback(log_weights=True), early_stopping],
          verbose=0)

    
    y_pred = np.rint(model.predict(x_val_encoded)).ravel()

    # Evaluation Metrics
    fbeta = fbeta_score(y_val.values.ravel(), y_pred, beta=1, zero_division=1)
    precision = precision_score(y_val.values.ravel(), y_pred, zero_division=1)
    recall = recall_score(y_val.values.ravel(), y_pred, zero_division=1)
    acc = accuracy_score(y_val.values.ravel(), y_pred)
    g_mean = geometric_mean_score(y_val.values.ravel(), y_pred)

    # run.summary["Accuracy"] = acc
    # run.summary["Precision"] = precision
    # run.summary["Recall"] = recall
    # run.summary["F1"] = fbeta
    # run.summary["G-Mean"] = g_mean  
    # 
    metrics = { "val_accuracy": acc,
                "val_precision": precision,
                "val_recall": recall,
                "val_F1": fbeta,
                "val_G-Mean": g_mean
                }
    wandb.log(metrics) 

In [24]:
# See the source code in order to see other parameters
# https://github.com/wandb/client/tree/master/wandb/sweeps

In [25]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
                # layer_1 = 32,
                # layer_2 = 32,
                # learn_rate = 0.001,
                # batch_size = 512,
                # epoch = 300,
                # dropout1 = 0.9,
                # dropout2 = 0.9,
                # l2_1 = 0.0,
                # l2_2 = 0.0,
                # gradient_cliping = 2,
                # bath_norm = 1

sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'binary_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'layer_1': {
            'max': 48,
            'min': 8,
            'distribution': 'int_uniform',
        },
        'layer_2': {
            'max': 48,
            'min': 8,
            'distribution': 'int_uniform',
        },
        'learn_rate': {
            'min': -4,
            'max': -2,
            'distribution': 'log_uniform', #  log_uniform
        },
        'batch_size': {
            'values': [32,128,256,1024,4096]
        },
        'epoch': {
            'values': [200,300,500]
        },
        'dropout1': {
            'min': 0.3,
            'max': 0.99,
            'distribution': 'uniform',  
        },
        'dropout2': {
            'min': 0.3,
            'max': 0.99,
            'distribution': 'uniform',  
        },
        'l2_1': {
            'min': 0.0,
            'max': 0.01,
            'distribution': 'uniform',  
        },
        'l2_2': {
            'min': 0.0,
            'max': 0.01,
            'distribution': 'uniform',  
        },
        'gradient_cliping': {
            'values': [0.0,0.5,1.0]
        },
        'bath_norm': {
            'values': [0,1]
        }
    }
}

In [26]:
# Initialize a new sweep

sweep_id = wandb.sweep(sweep_config, entity="mlops_ivan", project="bank_mlp")

Retry attempt failed:
Traceback (most recent call last):
  File "c:\Users\franc\.conda\envs\colab2deploy\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "c:\Users\franc\.conda\envs\colab2deploy\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "c:\Users\franc\.conda\envs\colab2deploy\lib\http\client.py", line 1348, in getresponse
    response.begin()
  File "c:\Users\franc\.conda\envs\colab2deploy\lib\http\client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "c:\Users\franc\.conda\envs\colab2deploy\lib\http\client.py", line 277, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "c:\Users\franc\.conda\envs\colab2deploy\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "c:\Users\franc\.conda\envs\colab2deploy\lib\ssl.py"

Create sweep with ID: 77zi45j8
Sweep URL: https://wandb.ai/mlops_ivan/bank_mlp/sweeps/77zi45j8


In [27]:
# Initialize a new sweep
# Arguments:
#     – sweep_id: the sweep_id to run - this was returned above by wandb.sweep()
#     – function: function that defines your model architecture and trains it
wandb.agent(sweep_id = sweep_id, function=train_func,count=80)

wandb: Agent Starting Run: jrb6qpsu with config:
wandb: 	batch_size: 32
wandb: 	bath_norm: 1
wandb: 	dropout1: 0.3764606967112928
wandb: 	dropout2: 0.43456465419606594
wandb: 	epoch: 300
wandb: 	gradient_cliping: 1
wandb: 	l2_1: 0.0013257709243123972
wandb: 	l2_2: 0.004725145840956012
wandb: 	layer_1: 29
wandb: 	layer_2: 9
wandb: 	learn_rate: 0.07590628599699328
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Restoring model weights from the end of the best epoch: 33.
Epoch 133: early stopping
297/297 [==============================] - 0s 473us/step



epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
f1_score,█▂▁▄▄▂▁▁▂▁▁▁▂▂▃▂▂▂▁▁▃▁▂▁▁▁▂▂▁▁▂▁▂▃▂▁▁▁▁▁
loss,█▄▁▇▆▃▁▂▂▁▁▁▁▂▁▁▁▂▁▁▂▁▂▁▁▁▂▁▁▁▂▁▁▂▁▁▁▁▁▁
val_F1,▁
val_G-Mean,▁
val_accuracy,▁
val_f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▅▃▃▄▄▃▄▃▃▃▄▅▃▆▄▅▅▅▄▃▂▃▄▁▄█▄▂▃▃▄▂▃▄▃▆▂▃▂▃
val_precision,▁
val_recall,▁
best_epoch,32


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: wszndnh8 with config:
wandb: 	batch_size: 1024
wandb: 	bath_norm: 1
wandb: 	dropout1: 0.5764855342654719
wandb: 	dropout2: 0.5561511812829791
wandb: 	epoch: 300
wandb: 	gradient_cliping: 0
wandb: 	l2_1: 0.000573599580436599
wandb: 	l2_2: 0.0010725526861955104
wandb: 	layer_1: 18
wandb: 	layer_2: 32
wandb: 	learn_rate: 0.11901904138115636
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Restoring model weights from the end of the best epoch: 8.
Epoch 108: early stopping
297/297 [==============================] - 0s 466us/step



...

wandb: Agent Starting Run: 8bvx23hb with config:
wandb: 	batch_size: 128
wandb: 	bath_norm: 1
wandb: 	dropout1: 0.30920958064683846
wandb: 	dropout2: 0.9803582086918188
wandb: 	epoch: 500
wandb: 	gradient_cliping: 0
wandb: 	l2_1: 0.00441031888667433
wandb: 	l2_2: 0.00402125113039913
wandb: 	layer_1: 24
wandb: 	layer_2: 44
wandb: 	learn_rate: 0.03971447505694739
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Restoring model weights from the end of the best epoch: 90.
Epoch 190: early stopping
297/297 [==============================] - 0s 434us/step



epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▂▁▁▁▂▁▂▁▁▁▁▃▄▁▁▁▁▁▂▂▁▁▁▁▂▁▁▂▁▂▁▁▁▂▂▁▁▁▁█
loss,▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▃▂▂▂▁▁▁█▃▂▂▁▁▁▁▁▁▁▁▁▁▁
val_F1,▁
val_G-Mean,▁
val_accuracy,▁
val_f1_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁
val_loss,▄▅▃▃▆▄▅▄▅▄▃▅▂▅▂▄▄▄▄▄▅▂▆▅▂█▆▅▂▄▂▅▄▅▃▄▄▅▅▁
val_precision,▁
val_recall,▁
best_epoch,89


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: f052lm80 with config:
wandb: 	batch_size: 4096
wandb: 	bath_norm: 1
wandb: 	dropout1: 0.8003525698738347
wandb: 	dropout2: 0.6618887811680236
wandb: 	epoch: 500
wandb: 	gradient_cliping: 0.5
wandb: 	l2_1: 0.00325207321324269
wandb: 	l2_2: 0.008832151776406054
wandb: 	layer_1: 10
wandb: 	layer_2: 24
wandb: 	learn_rate: 0.03037777163149354
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Restoring model weights from the end of the best epoch: 117.
Epoch 217: early stopping
297/297 [==============================] - 0s 471us/step



In [ ]:
run.finish()


In [ ]:
b=testacas

NameError: name 'testacas' is not defined

In [ ]:
# sweep_config = {
#     # try grid or random
#     "method": "random",
#     "metric": {
#         "name": "Accuracy",
#         "goal": "maximize"
#         },
#     "parameters": {
#         "criterion": {
#             "values": ["gini","entropy"]
#             },
#         "splitter": {
#             "values": ["random","best"]
#             },
#         "max_depth": {
#             "values": [5, 7, 9]
#             },
#         "model": {
#             "values": [0,1,2]
#         },
#         "random_state": {
#             "values": [seed]
#         }
#     }
# }

# sweep_id = wandb.sweep(sweep_config, project="bank_mlp", entity="mlops_ivan")

In [ ]:
# def train():
#     with wandb.init() as run:

#         # The full pipeline 
#         pipe = Pipeline(steps = [('full_pipeline', full_pipe),
#                                     ("classifier",DecisionTreeClassifier())
#                                     ]
#                         )

#         # update the parameters of the pipeline that we would like to tuning
#         pipe.set_params(**{"full_pipeline__num_pipeline__num_transformer__model": run.config.model})
#         pipe.set_params(**{"classifier__criterion": run.config.criterion})
#         pipe.set_params(**{"classifier__splitter": run.config.splitter})
#         pipe.set_params(**{"classifier__max_depth": run.config.max_depth})
#         pipe.set_params(**{"classifier__random_state": run.config.random_state})

#         # training
#         logger.info("Training")
#         pipe.fit(x_train, y_train)

#         # predict
#         logger.info("Infering")
#         predict = pipe.predict(x_val)

#         # Evaluation Metrics
#         logger.info("Evaluation metrics")
#         fbeta = fbeta_score(y_val, predict, beta=1, zero_division=1)
#         precision = precision_score(y_val, predict, zero_division=1)
#         recall = recall_score(y_val, predict, zero_division=1)
#         acc = accuracy_score(y_val, predict)

#         logger.info("Accuracy: {}".format(acc))
#         logger.info("Precision: {}".format(precision))
#         logger.info("Recall: {}".format(recall))
#         logger.info("F1: {}".format(fbeta))

#         run.summary["Accuracy"] = acc
#         run.summary["Precision"] = precision
#         run.summary["Recall"] = recall
#         run.summary["F1"] = fbeta

In [ ]:
# wandb.agent(sweep_id, train, count=24)

### 4.2.1 Restore a model

Restore a file, such as a model checkpoint, into your local run folder to access in your script.

See [the restore docs](https://docs.wandb.com/library/restore) for more details.

In [ ]:
!wandb login

In [ ]:
# restore the raw model file "model-best.h5" from a specific run by user "ivanovitchm"
# in project "lesson04" from run "sqdv5ccj"
best_model = wandb.restore('model-best.h5', run_path="ivanovitchm/week06/cbwfq70j")

In [ ]:
# restore the model for tf.keras
model = tf.keras.models.load_model(best_model.name)

In [ ]:
# execute the loss and accuracy using the test dataset
loss_, acc_ = model.evaluate(x=test_x,y=test_y, batch_size=64)
print('Test loss: %.3f - acc: %.3f' % (loss_, acc_))

In [ ]:
# source: https://github.com/wandb/awesome-dl-projects/blob/master/ml-tutorial/EMNIST_Dense_Classification.ipynb
import seaborn as sns
from sklearn.metrics import confusion_matrix

predictions = np.greater_equal(model.predict(test_x),0.5).astype(int)
cm = confusion_matrix(y_true = test_y, y_pred = predictions)

plt.figure(figsize=(6,6));
sns.heatmap(cm, annot=True)
plt.savefig('confusion_matrix.png', bbox_inches='tight')
plt.show()

In [ ]:
wandb.init(project="week06")
wandb.log({"image_confusion_matrix": [wandb.Image('confusion_matrix.png')]})

In [ ]:
# visualize the images and instances with error
# ground-truth
print("Ground-truth\n",test_y[~np.equal(predictions,test_y)])

# predictions
print("Predictions\n",predictions[~np.equal(predictions,test_y)])

In [ ]:
# Images predicted as non-cat
fig, ax = plt.subplots(2,6,figsize=(10,6))
wrong_images = (~np.equal(predictions,test_y)).astype(int)
index = np.where(wrong_images == 1)[0]

for i,value in enumerate(index):
  ax[i//6,i%6].imshow(test_x[value].reshape(64,64,3))
plt.savefig('wrong_predictions.png', bbox_inches='tight')

In [ ]:
wandb.log({"wrong_predictions": [wandb.Image('wrong_predictions.png')]})

## 1.7 Configure and train the best model

In [ ]:
# Note that a new run is yet synced to last sweep run
# Just to check
# run = wandb.init()

<font color="red">Important</font> to restart the colab to unlink a new experiment (run) with the last ```sweep``` experiment. 

```
Runtime >> Factory reset runtime
```
> Re-run all cells except for: ```1.4.3```, ```1.4.4```, ```1.5``` and ```1.6```. 

In [ ]:
# initiate the wandb project
run = wandb.init(project="bank_mlp", entity="mlops_ivan",job_type="train")

In [ ]:
# The full pipeline 
pipe = Pipeline(steps = [('full_pipeline', full_pipe),
                         ("classifier",DecisionTreeClassifier())
                         ]
                )

# update the parameters of the pipeline that we would like to tuning
# pipe.set_params(**{"full_pipeline__num_pipeline__num_transformer__model": 1})
# pipe.set_params(**{"classifier__criterion": 'entropy'})
# pipe.set_params(**{"classifier__splitter": 'random'})
# pipe.set_params(**{"classifier__max_depth": 6})
# pipe.set_params(**{"classifier__random_state": 41})

pipe.set_params(**{"full_pipeline__num_pipeline__num_transformer__model": 2})
pipe.set_params(**{"classifier__criterion": 'gini'})
pipe.set_params(**{"classifier__splitter": 'best'})
pipe.set_params(**{"classifier__max_depth": 6})
pipe.set_params(**{"classifier__class_weight": {0:1.0, 1:6.0}})
pipe.set_params(**{"classifier__random_state": 41})



# training
logger.info("Training")
pipe.fit(x_train, y_train)

# predict
logger.info("Infering")
predict = pipe.predict(x_val)

# Evaluation Metrics
logger.info("Evaluation metrics")
fbeta = fbeta_score(y_val, predict, beta=1, zero_division=1)
precision = precision_score(y_val, predict, zero_division=1)
recall = recall_score(y_val, predict, zero_division=1)
acc = accuracy_score(y_val, predict)

logger.info("Accuracy: {}".format(acc))
logger.info("Precision: {}".format(precision))
logger.info("Recall: {}".format(recall))
logger.info("F1: {}".format(fbeta))

run.summary["Acc"] = acc
run.summary["Precision"] = precision
run.summary["Recall"] = recall
run.summary["F1"] = fbeta

In [ ]:
# Get categorical column names
cat_names = pipe.named_steps['full_pipeline'].get_params()["cat_pipeline"][3].get_feature_names_out().tolist()
cat_names

In [ ]:
# Get numerical column names
num_names = pipe.named_steps['full_pipeline'].get_params()["num_pipeline"][2].get_feature_names_out()
num_names

In [ ]:
# merge all column names together
all_names = cat_names + num_names
all_names

In [ ]:
# Visualize all classifier plots
# For a complete documentation please see: https://docs.wandb.ai/guides/integrations/scikit
wandb.sklearn.plot_classifier(pipe.get_params()["classifier"],
                              full_pipe.transform(x_train),
                              full_pipe.transform(x_val),
                              y_train,
                              y_val,
                              predict,
                              pipe.predict_proba(x_val),
                              [0,1],
                              model_name='BestModel', feature_names=all_names)

## 1.8 Export the best model

In [ ]:
model.evaluate(testX, testY_one_hot)[1]

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc = 'upper left')

In [ ]:
# pipe = Pipeline(steps =  [
#                          ('full_pipeline', full_pipe),
#                          ("classifier", model)
#                          ])

In [ ]:
pipe = full_pipe

In [ ]:
# types and names of the artifacts
artifact_type = "inference_artifact"
artifact_encoder = "target_encoder"
artifact_model = "model_export"
artifact_keras_model = "keras_model_export"

In [ ]:
logger.info("Dumping the artifacts to disk")
# Save the pipiline preprocessing using joblib
joblib.dump(pipe, artifact_model)

# Save the target encoder using joblib
joblib.dump(le, artifact_encoder)

joblib.dump(model, artifact_keras_model)

In [ ]:
# Model artifact
artifact = wandb.Artifact(artifact_model,
                          type=artifact_type,
                          description="A full pipeline composed of a Preprocessing Stage and MLP bank marketing model"
                          )

logger.info("Logging model artifact")
artifact.add_file(artifact_model)
run.log_artifact(artifact)

In [ ]:
# Target encoder artifact
artifact = wandb.Artifact(artifact_encoder,
                          type=artifact_type,
                          description="The encoder used to encode the target variable"
                          )

logger.info("Logging target enconder artifact")
artifact.add_file(artifact_encoder)
run.log_artifact(artifact)

In [ ]:
# Model artifact
artifact = wandb.Artifact(artifact_keras_model,
                          type=artifact_type,
                          description="MLP Model for Classification Purpose"
                          )

logger.info("Logging keras model artifact")
model.save("path")
artifact.add_dir("path")
run.log_artifact(artifact)

In [ ]:
run.finish()

## Save keras model on run

In [ ]:
run = wandb.init(project="bank_mlp", entity="mlops_ivan",job_type="save")

In [ ]:
model.save(os.path.join(wandb.run.dir, "model.h5"))

In [ ]:
run.finish()

## Load Keras Model Restore

In [ ]:
import tensorflow as tf

best_model = wandb.restore('model.h5', run_path="mlops_ivan/bank_mlp/akrq1ozh")
# restore the model for tf.keras
model = tf.keras.models.load_model(best_model.name)

In [ ]:
pred = model.predict(x_train_encoded)
pred

## Load Keras Model Artifact

In [ ]:
import keras

In [ ]:
artifact_model_path = "mlops_ivan/bank_mlp/keras_model_export:latest"

# use the latest version of the model
model_at = run.use_artifact(artifact_model_path)
# download the directory in which the model is saved
model_dir= model_at.download()
print("model: ", model_dir)
model_t = keras.models.load_model(model_dir)

In [ ]:
model_t.predict(x_train_encoded)

## 1.5 Holdout Training

In this example the inference artifacts are:
- pipe (contains preprocesing pipeline)
- le (label encoder object used to encode the target variable)
- keras model (contains keras model artifact)

In [ ]:
def nn_model():	
  # define model
  model = Sequential()
  model.add(Dense(120, input_dim=32, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(60, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(1, activation='sigmoid'))

  # compile model
  opt =  keras.optimizers.Adam(learning_rate=0.01)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model

In [ ]:
# The full pipeline 

#early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')

early_stopping = callbacks.EarlyStopping(
                monitor='val_loss',   # 'loss'             
                min_delta=0.001, # Minimum change
                patience=50, # Number of epochs with no improvement
                verbose=1, # 
                mode='auto', # "auto", "min", "max"
                baseline=None, # Baseline value for the monitored quantity
                restore_best_weights=True) # restore weights epoch with best value

model = nn_model()


# training
logger.info("Training")
# fit model on train set
history = model.fit(x_train_encoded, y_train,
                    epochs=10,
                    verbose=1,
                    #batch_size=512,
                    validation_data=(x_val_encoded, y_val ),
                    callbacks=[early_stopping]
                    )




---



In [ ]:
# evaluate model on test set
_, test_acc = model.evaluate(full_pipe.transform(x_val), y_val, verbose=1)

In [ ]:
# Evaluation Metrics
def ful_pipe_predict(X):
    X_trans = full_pipe.transform(X)
    X_pred = model.predict(X_trans)
    X_pred_rint = np.rint(X_pred)
    return X_pred_rint

logger.info("Infering")
y_pred = ful_pipe_predict(x_val)


In [ ]:
# Evaluation Metrics

logger.info("Evaluation metrics")
fbeta = fbeta_score(y_val, y_pred, beta=1, zero_division=1)
precision = precision_score(y_val, y_pred, zero_division=1)
recall = recall_score(y_val, y_pred, zero_division=1)
acc = accuracy_score(y_val, y_pred)
g_mean = geometric_mean_score(y_val, y_pred)


logger.info("Accuracy: {}".format(acc))
logger.info("Precision: {}".format(precision))
logger.info("Recall: {}".format(recall))
logger.info("F1: {}".format(fbeta))
logger.info("G-Mean: {}".format(g_mean))

In [ ]:
# Compare the accuracy, precision, recall with previous ones
print(classification_report(y_val, y_pred))

In [ ]:
model.summary() # mostra modelo

In [ ]:
run.summary["Acc"] = acc
run.summary["Precision"] = precision
run.summary["Recall"] = recall
run.summary["F1"] = fbeta

In [ ]:
fig_confusion_matrix, ax = plt.subplots(1,1,figsize=(7,4))
ConfusionMatrixDisplay(confusion_matrix(y_pred,y_val,labels=[1,0]),
                       display_labels=["yes","no"]).plot(values_format=".0f",ax=ax)

ax.set_xlabel("True Label")
ax.set_ylabel("Predicted Label")
plt.show()

In [ ]:
# Uploading figures
logger.info("Uploading figures")
run.log(
    {
        "confusion_matrix": wandb.Image(fig_confusion_matrix),
        # "other_figure": wandb.Image(other_fig)
    }
)

In [ ]:
# close the current run before to execute the next section
run.finish()

# 2.0 Test evaluation

## 2.1 Evaluation

In [ ]:
# global variables

# name of the artifact related to test dataset
artifact_test_name = "mlops_ivan/bank_mlp/test.csv:latest"

# name of the model artifact
artifact_model_name =  "mlops_ivan/bank_mlp/model_export:latest"

# name of the target encoder artifact
artifact_encoder_name =  "mlops_ivan/bank_mlp/target_encoder:latest"

In [ ]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

In [ ]:
# initiate the wandb project
run = wandb.init(project="bank_mlp", entity="mlops_ivan",job_type="test")

In [ ]:
logger.info("Downloading and reading test artifact")
test_data_path = run.use_artifact(artifact_test_name).file()
df_test = pd.read_csv(test_data_path)

# Extract the target from the features
logger.info("Extracting target from dataframe")
x_test = df_test.copy()
y_test = x_test.pop("y")

In [ ]:
# Takes a look at test set
x_test.head()

In [ ]:
# Take a look at the target variable
y_test.head()

In [ ]:
# Extract the encoding of the target variable
logger.info("Extracting the encoding of the target variable")
encoder_export_path = run.use_artifact(artifact_encoder_name).file()
le = joblib.load(encoder_export_path)

In [ ]:
# transform y_train
y_test = le.transform(y_test)
logger.info("Classes [0, 1]: {}".format(le.inverse_transform([0, 1])))

In [ ]:
# target variable after the encoding
y_test

In [ ]:
# Download inference artifact
logger.info("Downloading and load the exported model")
model_export_path = run.use_artifact(artifact_model_name).file()
pipe = joblib.load(model_export_path)

In [ ]:
# predict
logger.info("Infering")
predict = pipe.predict(x_test)

# Evaluation Metrics
logger.info("Test Evaluation metrics")
fbeta = fbeta_score(y_test, predict, beta=1, zero_division=1)
precision = precision_score(y_test, predict, zero_division=1)
recall = recall_score(y_test, predict, zero_division=1)
acc = accuracy_score(y_test, predict)

logger.info("Test Accuracy: {}".format(acc))
logger.info("Test Precision: {}".format(precision))
logger.info("Test Recall: {}".format(recall))
logger.info("Test F1: {}".format(fbeta))

run.summary["Acc"] = acc
run.summary["Precision"] = precision
run.summary["Recall"] = recall
run.summary["F1"] = fbeta

In [ ]:
# Compare the accuracy, precision, recall with previous ones
print(classification_report(y_test,predict))

In [ ]:
fig_confusion_matrix, ax = plt.subplots(1,1,figsize=(7,4))
ConfusionMatrixDisplay(confusion_matrix(predict,y_test,labels=[1,0]),
                       display_labels=[">50k","<=50k"]).plot(values_format=".0f",ax=ax)

ax.set_xlabel("True Label")
ax.set_ylabel("Predicted Label")
plt.show()

In [ ]:
# Uploading figures
logger.info("Uploading figures")
run.log(
    {
        "confusion_matrix": wandb.Image(fig_confusion_matrix),
        # "other_figure": wandb.Image(other_fig)
    }
)

In [ ]:
run.finish()